In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install openpyxl

In [ ]:
dados = pd.read_excel('/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

In [ ]:
dados.head()

In [ ]:
x_columns = dados.describe().columns
dados_limpos = dados.dropna()

In [ ]:
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU"], axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# np.random.seed(73246)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=73246)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
modelo = LogisticRegression(solver='liblinear')
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

In [ ]:
acuracia_por_tamanho_teste = {}
for i in range(5,50,5):
    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=i/100, 
                                                        random_state=73246)
    modelo = LogisticRegression(solver='liblinear')
    modelo.fit(x_train, y_train)
    y_prediction = modelo.predict(x_test)
    acuracia_por_tamanho_teste[i] = accuracy_score(y_test, y_prediction)
acuracia_por_tamanho_teste

In [ ]:
acuracia_por_tamanho_teste = {}
for i in range(5,50,5):
    np.random.seed(3508743)
    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=i/100)
    modelo = LogisticRegression(solver='liblinear')
    modelo.fit(x_train, y_train)
    y_prediction = modelo.predict(x_test)
    acuracia_por_tamanho_teste[i] = accuracy_score(y_test, y_prediction)
acuracia_por_tamanho_teste

In [ ]:
janelas_lista = dados['WINDOW'].unique()
janelas_dict = {}
i = 0
for janela in janelas_lista:
    janelas_dict[janela] = i
    i += 1
print (janelas_lista)
print (janelas_dict)

In [ ]:
dados['COD_WINDOW'] = dados['WINDOW'].map(janelas_dict)
dados.head()

In [ ]:
pacientes_UTI = dados[['PATIENT_VISIT_IDENTIFIER','COD_WINDOW',
                       'ICU']].query('ICU == 1').groupby('PATIENT_VISIT_IDENTIFIER').min()
pacientes_UTI.columns = ['COD_WINDOW_INIC','ICU']
pacientes_UTI

In [ ]:
dados_tratados = dados.query('ICU != 1').drop('ICU', axis=1)
dados_tratados = dados_tratados.join(pacientes_UTI, on='PATIENT_VISIT_IDENTIFIER', how='left')
dados_tratados['DIF_COD_WINDOW'] = dados_tratados['COD_WINDOW_INIC'] - dados_tratados['COD_WINDOW']
dados_tratados.query('DIF_COD_WINDOW < 0')

In [ ]:
dados_tratados.drop(['DIF_COD_WINDOW','COD_WINDOW_INIC','COD_WINDOW'], axis=1, inplace=True)
dados_tratados['ICU'] = dados_tratados['ICU'].fillna(0) 
print (dados_tratados['ICU'].value_counts())
dados_tratados.head()

In [ ]:
percentil_idade = dados_tratados['AGE_PERCENTIL'].unique()
percentil_idade.sort()
perc_idade_dict = {}
i = 0
for item in percentil_idade:
    perc_idade_dict[item] = i
    i += 10
perc_idade_dict

In [ ]:
dados_tratados['AGE_PERCENTIL'] = dados_tratados['AGE_PERCENTIL'].map(perc_idade_dict)
dados_tratados.head()

In [ ]:
features_continuas_colunas = dados_tratados.iloc[:, 13:-2].columns
features_continuas = dados_tratados.groupby("PATIENT_VISIT_IDENTIFIER",as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
features_categoricas = dados_tratados.iloc[:, :13]
saida = dados_tratados.iloc[:, -2:]
dados_limpos = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
dados_limpos.columns = dados_tratados.columns
dados_limpos.describe()

In [ ]:
dados_limpos.dropna(inplace=True)
dados_limpos.describe()

In [ ]:
dados_limpos.head()

In [ ]:
dados_final = dados_limpos.loc[dados_limpos["WINDOW"] == "0-2"]
dados_final.reset_index(drop=True, inplace=True)
dados_final.head()

In [ ]:
y = dados_final["ICU"]
x_columns = dados_final.describe().columns[1:-1]
x = dados_final[x_columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=73246)
modelo = LogisticRegression(solver='liblinear')
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

In [ ]:
descricao = dados_final.describe().T
descricao['eliminar'] = descricao['min'] == descricao['max']
eliminacao = list(descricao.query('eliminar').index)
eliminacao

In [ ]:
dados_final = dados_final.drop(eliminacao, axis=1)
dados_final.head()

In [ ]:
y = dados_final["ICU"]
x_columns = dados_final.describe().columns[1:-1]
x = dados_final[x_columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=73246)
modelo = LogisticRegression(solver='liblinear')
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)